In [309]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [310]:
import numpy as np
import pandas as pd
import os

In [311]:
from sklearn import linear_model
from scipy import stats
from sklearn.decomposition import PCA

In [312]:
df = pd.read_excel('../data/q_oci.xlsx')

In [313]:
df.head(5)

,团队ID,职能,发展状态,负责人绩效,负责人管理角色,管理层级,年龄,团队工作时长,工作经验年限,员工类型,...,24,Unnamed: 68,26,Unnamed: 70,53,Unnamed: 72,54,Unnamed: 74,Unnamed: 75,Unnamed: 76
0,504,投资,初创期（业务刚起步）,5,业务负责人,基层管理者,28,0.3,4.0,被测评,...,4,5,4,5,3,5,5,5,NaN,NaN
1,504,投资,初创期（业务刚起步）,5,业务负责人,员工,25,2.0,2.0,测评,...,5,4,6,6,4,5,6,6,NaN,NaN
2,504,投资,初创期（业务刚起步）,5,业务负责人,员工,25,0.3,2.0,测评,...,6,5,6,6,6,6,6,6,NaN,NaN
3,504,投资,初创期（业务刚起步）,5,业务负责人,员工,28,0.9,2.0,测评,...,4,5,4,5,4,5,4,5,NaN,NaN
4,504,投资,初创期（业务刚起步）,5,业务负责人,员工,36,0.3,7.5,测评,...,4,4,4,4,4,4,4,4,NaN,NaN


In [314]:
df_person = df[df[u'员工类型'] == '测评']

In [315]:
df_person.iloc[:10, :15]

,团队ID,职能,发展状态,负责人绩效,负责人管理角色,管理层级,年龄,团队工作时长,工作经验年限,员工类型,寻找机会7,寻找机会8,启发灵感9,启发灵感10,启发灵感11
1,504,投资,初创期（业务刚起步）,5,业务负责人,员工,25,2.0,2.0,测评,6,5,6,6,5
2,504,投资,初创期（业务刚起步）,5,业务负责人,员工,25,0.3,2.0,测评,5,4,6,6,5
3,504,投资,初创期（业务刚起步）,5,业务负责人,员工,28,0.9,2.0,测评,4,4,4,4,4
4,504,投资,初创期（业务刚起步）,5,业务负责人,员工,36,0.3,7.5,测评,4,4,4,4,4
5,504,投资,初创期（业务刚起步）,5,业务负责人,员工,28,0.3,2.0,测评,4,3,4,5,5
7,505,运营,初创期（业务刚起步）,5,业务负责人,员工,28,0.3,2.0,测评,6,2,4,6,5
8,505,运营,初创期（业务刚起步）,5,业务负责人,员工,28,0.3,15.0,测评,3,4,4,1,3
9,505,运营,初创期（业务刚起步）,5,业务负责人,员工,25,0.3,0.3,测评,5,6,5,4,4
10,505,运营,初创期（业务刚起步）,5,业务负责人,员工,25,0.9,2.0,测评,6,1,6,6,5
11,505,运营,初创期（业务刚起步）,5,业务负责人,员工,25,0.3,7.5,测评,6,6,5,6,2


In [316]:
df_1 = df_person[[u'团队ID', u'寻找机会7', u'寻找机会8', u'启发灵感9', u'启发灵感10', u'启发灵感11', 44, 45, 46, 47, 48, 49, 50, 51, 52]]

In [317]:
df_1[:10]

,团队ID,寻找机会7,寻找机会8,启发灵感9,启发灵感10,启发灵感11,44,45,46,47,48,49,50,51,52
1,504,6,5,6,6,5,5,5,3,6,5,4,5,5,4
2,504,5,4,6,6,5,3,5,5,5,6,6,5,5,6
3,504,4,4,4,4,4,4,4,4,4,4,4,4,4,4
4,504,4,4,4,4,4,4,4,4,4,4,4,4,4,4
5,504,4,3,4,5,5,4,4,4,4,4,4,4,4,5
7,505,6,2,4,6,5,4,6,6,6,6,4,6,4,6
8,505,3,4,4,1,3,5,6,6,2,6,6,6,6,6
9,505,5,6,5,4,4,4,5,4,5,6,3,4,4,4
10,505,6,1,6,6,5,5,5,3,2,3,4,5,4,4
11,505,6,6,5,6,2,2,5,6,4,3,5,2,4,4


In [318]:
team_ids = np.unique(df_1[u'团队ID'])

In [319]:
team_ids

array([504, 505, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642,
       643, 644, 645, 646, 647, 648, 649, 651, 652, 653, 654, 655, 656,
       657, 658, 659, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670,
       671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 682, 683, 684,
       688, 689, 690, 699, 706, 707, 708, 709, 710, 711, 712, 714, 716,
       717, 718, 719, 720, 721, 722, 724, 725, 726, 727, 728, 740, 744],
      dtype=int64)

In [320]:
# 查看每个team里面成员的个数
num_team_person = df_1.groupby(u'团队ID').size()
num_team_person
# np.histogram(num_team_person)

团队ID
504     5
505     9
632     5
633     5
634     6
635     6
636     5
637     6
638     6
639     5
640     5
641     5
642     5
643     5
644     5
645     5
646     5
647     5
648     5
649     5
651     5
652     5
653     5
654     5
655     5
656     5
657     5
658     5
659     5
661     7
       ..
680     5
682     5
683     5
684     5
688     7
689     5
690     7
699    12
706     7
707     5
708     6
709     8
710     5
711     6
712     8
714     7
716     6
717     5
718     5
719     5
720     9
721    13
722    14
724     5
725     9
726     8
727     5
728     8
740     5
744     8
Length: 78, dtype: int64

In [321]:
avg_team_data = []
for team_id in team_ids:
    data_tmp = df_1[df_1[u'团队ID'] == team_id]
    data_tmp = np.mean(data_tmp, axis=0)[1:]
    avg_team_data.append(data_tmp)
avg_team_data = np.array(avg_team_data)

In [322]:
avg_team_data[:2, :]

array([[4.6       , 4.        , 4.8       , 5.        , 4.6       ,
        4.        , 4.4       , 4.        , 4.6       , 4.6       ,
        4.4       , 4.4       , 4.4       , 4.6       ],
       [5.22222222, 3.55555556, 5.        , 5.        , 4.33333333,
        4.11111111, 5.        , 4.77777778, 4.33333333, 4.77777778,
        4.66666667, 4.77777778, 4.22222222, 4.88888889]])

In [323]:
score_all = np.mean(avg_team_data[:, 5:], axis=1, keepdims=True)
score_1 = np.mean(avg_team_data[:, 5:8], axis=1, keepdims=True)
score_2 = np.mean(avg_team_data[:, 8:11], axis=1, keepdims=True)
score_3 = np.mean(avg_team_data[:, 11:14], axis=1, keepdims=True)
ques_score_all = np.concatenate([avg_team_data[:, :5], score_all], axis=1)
ques_score_1 = np.concatenate([avg_team_data[:, :5], score_1], axis=1)
ques_score_2 = np.concatenate([avg_team_data[:, :5], score_2], axis=1)
ques_score_3 = np.concatenate([avg_team_data[:, :5], score_3], axis=1)

In [324]:
# 前5列为5个问题的打分，最后1列为oci
ques_score_all[:10, :]

array([[4.6       , 4.        , 4.8       , 5.        , 4.6       ,
        4.37777778],
       [5.22222222, 3.55555556, 5.        , 5.        , 4.33333333,
        4.61728395],
       [5.2       , 3.2       , 4.4       , 4.8       , 4.        ,
        4.73333333],
       [4.4       , 4.2       , 5.4       , 5.6       , 5.2       ,
        4.91111111],
       [5.        , 3.66666667, 5.        , 5.33333333, 5.        ,
        4.68518519],
       [5.16666667, 2.5       , 4.66666667, 5.16666667, 5.5       ,
        4.18518519],
       [5.        , 3.2       , 5.        , 5.2       , 4.8       ,
        4.82222222],
       [4.33333333, 3.66666667, 3.83333333, 4.83333333, 4.        ,
        3.74074074],
       [4.16666667, 4.        , 3.83333333, 4.66666667, 4.33333333,
        4.2962963 ],
       [5.2       , 4.        , 5.        , 5.2       , 4.4       ,
        4.86666667]])

In [325]:
# 回归分析
def regression(func, x, y, alpha):
    regr = func(alpha=alpha)
    regr.fit(x, y)
    print('when use ' + str(func))
    print('w', regr.coef_ / np.sum(np.abs(regr.coef_) ))
    print('b', regr.intercept_)
    print('residual sum', np.mean((regr.predict(x) - y) ** 2))
    return regr.coef_

In [326]:
stats.zscore(y)

array([-0.04304654,  0.28613731,  0.36095182,  0.76495017,  0.47317358,
       -0.42460054,  0.63028405, -1.77126172, -0.64904407,  0.89961629,
        0.49561793,  0.76495017,  0.89961629, -0.31237877, -0.17771266,
        0.36095182, -0.17771266, -0.17771266, -1.52437384,  1.16894852,
        0.89961629,  0.63028405, -0.44704489,  0.09161958,  0.2262857 ,
        0.2262857 , -0.85104325,  0.89961629, -0.04304654,  0.74571215,
       -0.04304654, -0.42460054, -1.2550416 ,  0.49561793, -0.04304654,
        0.76495017,  1.03428241, -0.40856886, -3.40969949,  0.36095182,
       -2.06303831,  0.2262857 ,  0.89961629,  0.36095182, -0.58171101,
        0.63028405,  0.36095182,  0.13650829, -1.38970772,  0.09161958,
        0.2262857 ,  0.2262857 , -0.69713911, -2.19770443, -1.65903995,
       -1.26626378, -0.50475894, -0.85104325,  0.02428652,  0.52928446,
       -0.71637713,  0.58539535,  0.44511814, -0.69713911,  0.58539535,
        1.30361464,  0.09161958, -2.33237054,  1.78242751,  1.29

In [327]:
for ques_score in [ques_score_all, ques_score_1, ques_score_2, ques_score_3]:
    x = ques_score[:, :5]
    y = ques_score[:, 5]
    # x = stats.zscore(x, axis=0)
    # y = stats.zscore(y)
    regression(linear_model.Ridge, x, y, 0.1)
#     regression(linear_model.Lasso, x, y, 0.1)
    print("=======================================================")

when use <class 'sklearn.linear_model.ridge.Ridge'>
w [0.19827086 0.15294908 0.16529714 0.39713252 0.0863504 ]
b 1.4495666885948406
residual sum 0.10223688988334365
when use <class 'sklearn.linear_model.ridge.Ridge'>
w [ 0.22728676  0.12717166  0.12059996  0.50177452 -0.0231671 ]
b 1.8332408856956453
residual sum 0.11869641016148286
when use <class 'sklearn.linear_model.ridge.Ridge'>
w [0.18196649 0.24743388 0.17570714 0.17014713 0.22474537]
b 1.3692789181763514
residual sum 0.1058493431223344
when use <class 'sklearn.linear_model.ridge.Ridge'>
w [0.18263669 0.07313861 0.18591163 0.52316961 0.03514346]
b 1.1461802619125288
residual sum 0.14232759501546727


In [328]:
# 主成分回归
pca = PCA(n_components=2)
pca.fit(x)
print('lambda', str(pca.explained_variance_ratio_))
x_t = pca.transform(x)
w = regression(linear_model.Ridge, x_t, y, 0.05)
space_pca = np.diag(w)
space_x = pca.inverse_transform(space_pca)
print(space_x)
weight = np.sum(space_x, axis=0)
weight

lambda [0.55170692 0.29722959]
when use <class 'sklearn.linear_model.ridge.Ridge'>
w [-0.82911912 -0.17088088]
b 4.487976920989742
residual sum 0.14684819712131933
[[5.09503348 3.87269471 4.97777964 5.27448661 4.79651118]
 [4.92939141 3.91645167 4.82241478 5.12917168 4.63139837]]


array([10.02442489,  7.78914639,  9.80019442, 10.40365829,  9.42790955])

### 第二问

In [333]:
df_2 = df_person[[u'团队ID', u'职能', u'发展状态', u'负责人绩效', u'负责人管理角色', u'年龄', u'团队工作时长', u'工作经验年限', u'寻找机会7', u'寻找机会8', u'启发灵感9', u'启发灵感10', u'启发灵感11', 44, 45, 46, 47, 48, 49, 50, 51, 52]]

In [337]:
df_2.iloc[:5, :]

,团队ID,职能,发展状态,负责人绩效,负责人管理角色,年龄,团队工作时长,工作经验年限,寻找机会7,寻找机会8,...,启发灵感11,44,45,46,47,48,49,50,51,52
1,504,投资,初创期（业务刚起步）,5,业务负责人,25,2.0,2.0,6,5,...,5,5,5,3,6,5,4,5,5,4
2,504,投资,初创期（业务刚起步）,5,业务负责人,25,0.3,2.0,5,4,...,5,3,5,5,5,6,6,5,5,6
3,504,投资,初创期（业务刚起步）,5,业务负责人,28,0.9,2.0,4,4,...,4,4,4,4,4,4,4,4,4,4
4,504,投资,初创期（业务刚起步）,5,业务负责人,36,0.3,7.5,4,4,...,4,4,4,4,4,4,4,4,4,4
5,504,投资,初创期（业务刚起步）,5,业务负责人,28,0.3,2.0,4,3,...,5,4,4,4,4,4,4,4,4,5


In [393]:
avg_team_data = []
group_data = []
for team_id in team_ids:
    data_tmp = df_2[df_2[u'团队ID'] == team_id]
    data_tmp = np.mean(data_tmp.iloc[:, 5:], axis=0)
    avg_team_data.append(data_tmp)
    info = df_2[df_2[u'团队ID'] == team_id].iloc[0, :5]
    group_data.append(info)
avg_team_data = np.array(avg_team_data)
avg_team_data[:, 0] = avg_team_data[:, 0] // 5
avg_team_data[:, 1] = avg_team_data[:, 1] // 1
avg_team_data[:, 2] = avg_team_data[:, 2] // 3
group_data = np.array(group_data)

In [394]:
info_ques_score = np.concatenate([group_data, avg_team_data], axis=1)
info_ques_score[:2, :]
score_mean = np.mean(info_ques_score[:, -9:], axis=1, keepdims=True)
info_ques_mean_score = np.concatenate([info_ques_score[:, :-9], score_mean], axis=1)
info_ques_mean_score[:3, :]

array([[504, '投资', '初创期（业务刚起步）', 5, '业务负责人', 5.0, 0.0, 1.0, 4.6, 4.0,
        4.8, 5.0, 4.6, 4.377777777777777],
       [505, '运营', '初创期（业务刚起步）', 5, '业务负责人', 5.0, 0.0, 1.0,
        5.222222222222222, 3.5555555555555554, 5.0, 5.0,
        4.333333333333333, 4.617283950617284],
       [632, '行政', '成熟期（业务发展稳定成熟）', 0, '单一模块职能负责人', 5.0, 1.0, 2.0, 5.2,
        3.2, 4.4, 4.8, 4.0, 4.7333333333333325]], dtype=object)

In [399]:
def oci_distri(col_name):
    print(col_name)
    name_idx = {u'职能': 1, u'发展状态': 2, u'负责人绩效': 3, u'负责人管理角色': 4, u'年龄': 5, u'团队工作时长': 6, u'工作经验年限': 7}
    valid_col = np.array([name_idx[col_name], 8, 9, 10, 11, 12, 13])
    col_oci = info_ques_mean_score[:, valid_col]
    unique_labels = np.unique(col_oci[:, 0])
    label_score = {}
    for label in unique_labels:
        label_data = np.array(col_oci[col_oci[:, 0] == label][:, 1:])
        cov = []
        for i in range(5):
            cov.append(np.corrcoef(np.array(label_data[:, i], dtype=np.float), np.array(label_data[:, 5], dtype=np.float))[0, 1])
        label_score[label] = np.mean(np.abs(cov))
    print(label_score)
    print('===========================================================================================================')

In [412]:
info_ques_mean_score[0, :]

array([504, '投资', '初创期（业务刚起步）', 5, '业务负责人', 5.0, 0.0, 1.0, 4.6, 4.0, 4.8,
       5.0, 4.6, 4.377777777777777], dtype=object)

In [417]:
info_ques_mean_score[:, 7]

array([1.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 1.0, 4.0, 1.0, 2.0,
       1.0, 1.0, 1.0, 3.0, 4.0, 3.0, 4.0, 2.0, 1.0, 0.0, 2.0, 1.0, 0.0,
       3.0, 2.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.0, 2.0, 2.0, 0.0, 3.0, 3.0,
       1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 3.0, 2.0, 2.0,
       2.0, 0.0, 1.0, 1.0, 2.0, 1.0, 3.0, 2.0, 0.0, 3.0, 1.0, 2.0, 4.0,
       3.0, 5.0, 1.0, 3.0, 3.0, 4.0, 3.0, 4.0, 1.0, 1.0, 1.0, 2.0, 1.0],
      dtype=object)

In [400]:
for col_name in [u'职能', u'发展状态', u'负责人绩效', u'负责人管理角色', u'年龄', u'团队工作时长', u'工作经验年限']:
    oci_distri(col_name)

职能
{'HR': 0.2043266468386058, '产品': 0.5190360127306779, '内容（新闻/影视）': 0.4849405475955423, '客服': 1.0, '市场': nan, '技术': 0.6217352505301219, '投资': nan, '法务': nan, '综合职能': 0.6154399019463208, '行政': nan, '设计': 1.0, '财务': nan, '运营': 0.7206057980572067, '销售': 0.3411653697828938}
发展状态
{'初创期（业务刚起步）': 0.48960071593289767, '成熟期（业务发展稳定成熟）': 0.36663678340444267, '成长期（业务快速发展）': 0.5082415321833244}
负责人绩效
{0: 0.4689544582040822, 2: 0.6497422390351526, 3: 0.4507540384915375, 5: nan}
负责人管理角色
{'业务负责人': 0.5764659836803967, '单一模块职能负责人': 0.5486546845582975, '多模块职能负责人': 0.4017582960299664}
年龄
{5.0: 0.46505936421825683, 6.0: 0.5766619500070206, 7.0: 0.5317523428898292}
团队工作时长
{0.0: 0.3493567014916365, 1.0: 0.49000876048820424, 2.0: 0.5742900819158855}
工作经验年限
{0.0: 0.1769141937471106, 1.0: 0.63691312910622, 2.0: 0.4921169332954098, 3.0: 0.6821498293614214, 4.0: 0.5615726031814091, 5.0: nan}


C:\Tools\anaconda\lib\site-packages\numpy\lib\function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Tools\anaconda\lib\site-packages\numpy\lib\function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Tools\anaconda\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Tools\anaconda\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
C:\Tools\anaconda\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)
